In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [14]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import preprocess

In [15]:
data = pd.read_csv('../../Data/Sentimental_Analysis/main.csv',encoding='utf-8')

In [16]:
data.head(1)

,LABEL,TWEET
0,anger,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...


In [17]:
tweet = list(data["TWEET"])
label = list(data['LABEL'])

In [18]:
for i in range(len(tweet)):
    tweet[i] = preprocess.pre_process(str(tweet[i]))

In [19]:
vectorizer = TfidfVectorizer (encoding='utf-8', max_df=0.7)
tweet = vectorizer.fit_transform(tweet)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(tweet, label, test_size=0.1, random_state=0)

In [21]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_jobs=2, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(n_jobs=2, random_state=0)

In [22]:
predictions = text_classifier.predict(X_test)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[350  81 200 174 132]
 [ 63 498 184 174 130]
 [ 40  64 970 186 298]
 [ 88  87 310 870 162]
 [ 42  70 265 127 567]]
              precision    recall  f1-score   support

       anger       0.60      0.37      0.46       937
        fear       0.62      0.47      0.54      1049
         joy       0.50      0.62      0.56      1558
     sadness       0.57      0.57      0.57      1517
    surprise       0.44      0.53      0.48      1071

    accuracy                           0.53      6132
   macro avg       0.55      0.51      0.52      6132
weighted avg       0.54      0.53      0.53      6132

0.5308219178082192


In [24]:
text_classifier.fit(tweet, label)

RandomForestClassifier(n_jobs=2, random_state=0)

In [25]:
predictions = text_classifier.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[ 928    4    1    1    3]
 [   3 1037    0    3    6]
 [   0    2 1545    2    9]
 [   1    6    1 1507    2]
 [   2    9    5    2 1053]]
              precision    recall  f1-score   support

       anger       0.99      0.99      0.99       937
        fear       0.98      0.99      0.98      1049
         joy       1.00      0.99      0.99      1558
     sadness       0.99      0.99      0.99      1517
    surprise       0.98      0.98      0.98      1071

    accuracy                           0.99      6132
   macro avg       0.99      0.99      0.99      6132
weighted avg       0.99      0.99      0.99      6132

0.9898891063274625


In [36]:
text = input()
text = preprocess.pre_process(text)
print(text)
tweet = vectorizer.transform([text])
prediction = text_classifier.predict(tweet)
print(prediction)

انا عندي امتحان بكرا صعب
['sadness']


In [27]:
import pickle
filename = f'../../utils/sentmental_model.sav'
pickle.dump(text_classifier, open(filename, 'wb'))
filename = f'../../utils/tfidf_model.sav'
pickle.dump(vectorizer, open(filename, 'wb'))